In [ ]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import numpy as np
from keras.models import load_model
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.dataset import prepare_gtsf_data, compute_akpd_score
from weight_estimation.train import train, augment, normalize, get_data_split, train_model
from typing import Dict, Tuple


<h1> Load Raw GTSF Data </h1>

In [ ]:
s3 = S3AccessUtils('/root/data')

In [ ]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
akpd_scorer_f, _, _ = s3.download_from_url(akpd_scorer_url)
df1 = prepare_gtsf_data('2019-03-01', '2019-09-20', akpd_scorer_f, 0.5, 1.0)

In [ ]:
df2 = prepare_gtsf_data('2020-06-01', '2020-08-20', akpd_scorer_f, 0.5, 1.0)

In [ ]:
df = pd.concat([df1, df2])

<h1> Augment the Data </h1>

In [ ]:
from weight_estimation.utils import get_left_right_keypoint_arrs, get_ann_from_keypoint_arrs,\
    convert_to_nn_input, CameraMetadata

def augment(df: pd.DataFrame, augmentation_config: Dict) -> pd.DataFrame:
    print('hello')
    
    counts, edges = np.histogram(df.weight, bins=np.arange(0, 10000, 1000))
    trial_values = (5.0 / (counts / np.max(counts))).astype(int)
    max_jitter_std = augmentation_config['max_jitter_std']
    min_depth = augmentation_config['min_depth']
    max_depth = augmentation_config['max_depth']

    augmented_data = defaultdict(list)
    for idx, row in df.iterrows():
        
        camera_metadata = row.camera_metadata
        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )
        
        weight = row.weight
        trials = trial_values[min(int(weight / 1000), len(trial_values) - 1)]
        for _ in range(trials):
            
            ann = row.keypoints
            X_left, X_right = get_left_right_keypoint_arrs(ann)
            wkps = convert_to_world_point_arr(X_left, X_right, cm)
            original_depth = np.median(wkps[:, 1])
            
            depth = np.random.uniform(min_depth, max_depth)
            scaling_factor = float(original_depth) / depth
            jitter_std = np.random.uniform(0, max_jitter_std)
            

            # rescale
            X_left = X_left * scaling_factor
            X_right = X_right * scaling_factor

            # add jitter
            X_left[:, 0] += np.random.normal(0, jitter_std, X_left.shape[0])
            X_right[:, 0] += np.random.normal(0, jitter_std, X_right.shape[0])

            # reconstruct annotation
            ann = get_ann_from_keypoint_arrs(X_left, X_right)
            augmented_data['annotation'].append(ann)
            augmented_data['fish_id'].append(row.fish_id)
            augmented_data['weight'].append(row.weight)
            augmented_data['kf'].append(row.k_factor)
            augmented_data['camera_metadata'].append(row.camera_metadata)

    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df

In [ ]:
augmentation_config = dict(
    trials=10,
    max_jitter_std=10,
    min_depth=0.5,
    max_depth=2.5
)

augmented_df = augment(df, augmentation_config)

In [ ]:
augmented_df.to_csv('/root/data/alok/biomass_estimation/playground/augmented_df_depth_weight_balanced.csv')

<h3> Note: cell below takes about 1.5 hrs to run. To load cached version, run the cell below this one </h3>

In [ ]:
count = 0
akpd_scores = []
akpd_scorer_network = load_model(akpd_scorer_f)
for idx, row in augmented_df.iterrows():
    if count % 1000 == 0:
        print('Percentage complete: {}%'.format(round(100 * count / augmented_df.shape[0], 2)))
    count += 1
    akpd_score = compute_akpd_score(akpd_scorer_network, row.annotation, row.camera_metadata)
    akpd_scores.append(akpd_score)


augmented_df['akpd_score'] = akpd_scores

<h3> If you ran the cell above, do not run the one here </h3>

In [ ]:
augmented_df = pd.read_csv('/root/data/alok/biomass_estimation/playground/gtsf_augmented_dataset.csv')

new_anns, new_cms = [], []
for idx, row in augmented_df.iterrows():
    cm = row.camera_metadata
    new_cm = json.loads(cm.replace("'", '"'))
    new_cms.append(new_cm)
    
    ann = row.annotation
    new_ann = json.loads(ann.replace("'", '"'))
    new_anns.append(new_ann)
    
augmented_df['annotation'] = new_anns
augmented_df['camera_metadata'] = new_cms


<h1> Train model </h1>

In [ ]:
from collections import defaultdict
import json
import os
import random
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.interpolate import interpn
from weight_estimation.utils import get_left_right_keypoint_arrs, get_ann_from_keypoint_arrs,\
    convert_to_nn_input, CameraMetadata
from weight_estimation.dataset import prepare_gtsf_data
from keras.layers import Input, Dense, Flatten
from keras.models import Model
import keras
from research_lib.utils.data_access_utils import S3AccessUtils
import torch
from torch import nn
from sklearn.linear_model import LinearRegression




class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x
    
    def forward_intermediate(self, x):
        """Run inference on input keypoint tensor and get final hiddel layer weights."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x
        



def normalize(anns: List, camera_metadatas: List) -> np.ndarray:
    norm_anns = []
    for ann, camera_metadata in zip(anns, camera_metadatas):

        cm = CameraMetadata(
            focal_length=camera_metadata['focalLength'],
            focal_length_pixel=camera_metadata['focalLengthPixel'],
            baseline_m=camera_metadata['baseline'],
            pixel_count_width=camera_metadata['pixelCountWidth'],
            pixel_count_height=camera_metadata['pixelCountHeight'],
            image_sensor_width=camera_metadata['imageSensorWidth'],
            image_sensor_height=camera_metadata['imageSensorHeight']
        )

        norm_ann = convert_to_nn_input(ann, cm)
        norm_anns.append(norm_ann)
    return np.array(norm_anns)


def get_data_split(X: np.ndarray, y: np.ndarray, fish_ids: np.ndarray, train_pct: float,
                   val_pct: float) -> Tuple:
    # select train / test sets such that there are no overlapping fish IDs

    test_pct = 1.0 - train_pct - val_pct
    unique_fish_ids = np.array(list(set(fish_ids)))
    train_cnt, val_cnt, test_cnt = np.random.multinomial(len(unique_fish_ids),
                                                         [train_pct, val_pct, test_pct])

    assignments = np.array([0] * train_cnt + [1] * val_cnt + [2] * test_cnt)
    np.random.shuffle(assignments)
    train_fish_ids = unique_fish_ids[np.where(assignments == 0)]
    val_fish_ids = unique_fish_ids[np.where(assignments == 1)]
    test_fish_ids = unique_fish_ids[np.where(assignments == 2)]

    train_mask = np.isin(fish_ids, train_fish_ids)
    val_mask = np.isin(fish_ids, val_fish_ids)
    test_mask = np.isin(fish_ids, test_fish_ids)

    X_train, y_train = X[train_mask], y[train_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test, y_test = X[test_mask], y[test_mask]

    return X_train, y_train, X_val, y_val, X_test, y_test, train_mask, val_mask, test_mask


def train_model(X_train, y_train, X_val, y_val, train_config):
    inputs = Input(shape=(24,))
    x = Dense(256, activation='relu')(inputs)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    pred = Dense(1)(x)
    model = Model(inputs, pred)

    epochs = train_config['epochs']
    batch_size = train_config['batch_size']
    lr = train_config['learning_rate']
    patience = train_config['patience']

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=patience,
                                               verbose=0,
                                               mode='auto')]

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callbacks,
              batch_size=batch_size, epochs=epochs)

    return model


def convert_to_pytorch(model):
    pytorch_model = Network()
    weights = model.get_weights()

    pytorch_model.fc1.weight.data = torch.from_numpy(np.transpose(weights[0]))
    pytorch_model.fc1.bias.data = torch.from_numpy(np.transpose(weights[1]))
    pytorch_model.fc2.weight.data = torch.from_numpy(np.transpose(weights[2]))
    pytorch_model.fc2.bias.data = torch.from_numpy(np.transpose(weights[3]))
    pytorch_model.fc3.weight.data = torch.from_numpy(np.transpose(weights[4]))
    pytorch_model.fc3.bias.data = torch.from_numpy(np.transpose(weights[5]))
    pytorch_model.output.weight.data = torch.from_numpy(np.transpose(weights[6]))
    pytorch_model.output.bias.data = torch.from_numpy(np.transpose(weights[7]))
    
    return pytorch_model


def apply_final_layer_ols(pytorch_model):
    X_ols = pytorch_model.forward_intermediate(torch.from_numpy(X_train).float()).detach().numpy()
    lr = LinearRegression().fit(X_ols, y_train)
    pytorch_model.output.weight.data = torch.from_numpy(np.array(lr.coef_).reshape(1, -1))
    pytorch_model.output.bias.data = torch.from_numpy(np.array([lr.intercept_]))


In [ ]:
random.seed(0)
np.random.seed(0)
anns = augmented_df.annotation.values.tolist()
cms = augmented_df.camera_metadata.values.tolist()
X = normalize(anns, cms)

train_config = dict(
    train_pct=0.8,
    val_pct=0.1,
    epochs=500,
    batch_size=64,
    learning_rate=2e-5,
    patience=30
)

y = 1e-4 * augmented_df.weight.values
fish_ids = augmented_df.fish_id.values
X_train, y_train, X_val, y_val, X_test, y_test, train_mask, val_mask, test_mask = get_data_split(X, y, fish_ids,
                                                                train_config['train_pct'],
                                                                train_config['val_pct'])

tf_model = train_model(X_train, y_train, X_val, y_val, train_config)
pytorch_model = convert_to_pytorch(tf_model)
apply_final_layer_ols(pytorch_model)



In [ ]:
augmented_df['is_train'] = train_mask.astype(int)
augmented_df['is_val'] = val_mask.astype(int)
augmented_df['is_test'] = test_mask.astype(int)


In [ ]:
pytorch_model = convert_to_pytorch(tf_model)

<h1> Overall accuracy stats </h1>

In [ ]:
y_pred = (pytorch_model(torch.from_numpy(X).float())).detach().numpy().squeeze()
print('Train stats')
train_errs = (y_pred[train_mask] - y_train) / y_train
print('Mean error pct: {}'.format((np.mean(y_pred[train_mask]) - np.mean(y_train)) / np.mean(y_train)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(train_errs))))
print('='*20)
print('Val stats')
val_errs = (y_pred[val_mask] - y_val) / y_val
print('Mean error pct: {}'.format((np.mean(y_pred[val_mask]) - np.mean(y_val)) / np.mean(y_val)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(val_errs))))
print('='*20)
print('Test stats')
test_errs = (y_pred[test_mask] - y_test) / y_test
print('Mean error pct: {}'.format((np.mean(y_pred[test_mask]) - np.mean(y_test)) / np.mean(y_test)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(test_errs))))


In [ ]:
y_pred = (pytorch_model(torch.from_numpy(X).float())).detach().numpy().squeeze()
print('Train stats')
train_errs = (y_pred[train_mask] - y_train) / y_train
print('Mean error pct: {}'.format((np.mean(y_pred[train_mask]) - np.mean(y_train)) / np.mean(y_train)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(train_errs))))
print('='*20)
print('Val stats')
val_errs = (y_pred[val_mask] - y_val) / y_val
print('Mean error pct: {}'.format((np.mean(y_pred[val_mask]) - np.mean(y_val)) / np.mean(y_val)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(val_errs))))
print('='*20)
print('Test stats')
test_errs = (y_pred[test_mask] - y_test) / y_test
print('Mean error pct: {}'.format((np.mean(y_pred[test_mask]) - np.mean(y_test)) / np.mean(y_test)))
print('Mean absolute error pct: {}'.format(np.mean(np.abs(test_errs))))


In [ ]:
def generate_per_bucket_error(X, y):
    y_pred = (pytorch_model(torch.from_numpy(X).float())).detach().numpy().squeeze()
    print(np.mean(y_pred - y))

    buckets = np.arange(0, 10000, 1000) * 1e-4
    bucket_strs = []
    mean_errs = []
    maes = []
    for low, high in zip(buckets, buckets[1:]):
        bucket_str = '{}-{}'.format(round(1e4 * low), round(1e4 * high))
        mask = (y >= low) & (y < high)
        mean_err = np.mean((y_pred[mask] - y[mask]) / y[mask])
        mae = np.mean(np.abs((y_pred[mask] - y[mask]) / y[mask]))
        mean_errs.append(mean_err)
        maes.append(mae)
        bucket_strs.append(bucket_str)
    
    return pd.DataFrame({'bucket': bucket_strs, 'mean_err': mean_errs, 'mae': maes})

In [ ]:
generate_per_bucket_error(X_train, y_train)

In [ ]:
generate_per_bucket_error(X_train, y_train)

In [ ]:
generate_per_bucket_error(X_val, y_val)

In [ ]:
generate_per_bucket_error(X_val, y_val)

In [ ]:
generate_per_bucket_error(X_test, y_test)

In [ ]:
generate_per_bucket_error(X_test, y_test)

<h1> Generate errors with respect to depth </h1>

In [ ]:
# get depth array and add as column to augmented data-frame

from weight_estimation.utils import get_left_right_keypoint_arrs, convert_to_world_point_arr

depths = []
for idx, row in augmented_df.iterrows():
    ann, camera_metadata = row.annotation, row.camera_metadata
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    X = convert_to_world_point_arr(*get_left_right_keypoint_arrs(ann), cm)
    median_depth = np.median(X[:, 1])
    depths.append(median_depth)
    
augmented_df['depth'] = depths

In [ ]:
predictions = (pytorch_model(torch.from_numpy(X).float())).detach().numpy().squeeze()
augmented_df['y_pred'] = predictions
augmented_df['y'] = y

depths = np.arange(0.2, 2.7, 0.1)
mean_pct_errs = []
depth_buckets = []
for low_depth, high_depth in zip(depths, depths[1:]):
    depth_bucket = '{}-{}'.format(round(low_depth, 2), round(high_depth, 2))
    depth_buckets.append(depth_bucket)
    mask = (augmented_df.depth >= low_depth) & (augmented_df.depth <= high_depth) & train_mask
    mean_pct_err = np.mean((augmented_df[mask].y_pred - augmented_df[mask].y) / augmented_df[mask].y)
    mean_pct_errs.append(mean_pct_err)
    

pd.DataFrame({'depth_bucket': depth_buckets, 'mean_err': mean_pct_errs})

In [ ]:
predictions = (pytorch_model(torch.from_numpy(X).float())).detach().numpy().squeeze()
augmented_df['y_pred'] = predictions
augmented_df['y'] = y

depths = np.arange(0.2, 2.7, 0.1)
mean_pct_errs = []
depth_buckets = []
for low_depth, high_depth in zip(depths, depths[1:]):
    depth_bucket = '{}-{}'.format(round(low_depth, 2), round(high_depth, 2))
    depth_buckets.append(depth_bucket)
    mask = (augmented_df.depth >= low_depth) & (augmented_df.depth <= high_depth) & test_mask
    mean_pct_err = np.mean((augmented_df[mask].y_pred - augmented_df[mask].y) / augmented_df[mask].y)
    mean_pct_errs.append(mean_pct_err)
    

pd.DataFrame({'depth_bucket': depth_buckets, 'mean_err': mean_pct_errs})

In [ ]:
preds = pytorch_model(torch.from_numpy(X).float()).detach().numpy().squeeze()

In [ ]:
test_stats

In [ ]:
np.mean((pytorch_model(torch.from_numpy(X_test).float()).detach().numpy().squeeze() - y_test) / y_test)

In [ ]:
def generate_per_bucket_error_2(y_pred, y):

    buckets = np.arange(0, 10000, 1000) * 1e-4
    bucket_strs = []
    mean_errs = []
    for low, high in zip(buckets, buckets[1:]):
        bucket_str = '{}-{}'.format(round(1e4 * low), round(1e4 * high))
        mask = (y >= low) & (y < high)
        mean_err = np.mean((y_pred[mask] - y[mask]) / y[mask])
        mean_errs.append(mean_err)
        bucket_strs.append(bucket_str)
    
    return pd.DataFrame({'bucket': bucket_strs, 'mean_err': mean_errs})

In [ ]:
generate_per_bucket_error_2(preds, y)

In [ ]:
augmented_df.shape

In [ ]:
predictions = model(X)
augmented_df['y_pred_2'] = preds
augmented_df['y'] = y

depths = np.arange(0.2, 2.7, 0.1)
mean_pct_errs = []
depth_buckets = []
for low_depth, high_depth in zip(depths, depths[1:]):
    depth_bucket = '{}-{}'.format(round(low_depth, 2), round(high_depth, 2))
    depth_buckets.append(depth_bucket)
    mask = (augmented_df.depth >= low_depth) & (augmented_df.depth <= high_depth) & train_mask
    mean_pct_err = np.mean((augmented_df[mask].y_pred_2 - augmented_df[mask].y) / augmented_df[mask].y)
    mean_pct_errs.append(mean_pct_err)
    

pd.DataFrame({'depth_bucket': depth_buckets, 'mean_err': mean_pct_errs})

In [ ]:


def density_scatter(x, y, bins=20, **kwargs):
    fig, ax = plt.subplots(figsize=(20, 10))
    data, x_e, y_e = np.histogram2d(x, y, bins=bins, density=True)
    z = interpn((0.5*(x_e[1:] + x_e[:-1]), 0.5*(y_e[1:]+y_e[:-1])), data, np.vstack([x, y]).T,
                method="splinef2d", bounds_error=False)

    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    ax.scatter(x, y, c=z, **kwargs)

    norm = Normalize(vmin=np.min(z), vmax=np.max(z))
    cbar = fig.colorbar(cm.ScalarMappable(norm=norm), ax=ax)
    cbar.ax.set_ylabel('Density')

    ax.set_xlabel('Prediction')
    ax.set_ylabel('Ground Truth')
    ax.grid()

    return ax